<a href="https://colab.research.google.com/github/Onethybeing/pytorch_implementation/blob/main/FCN8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn.functional as F

In [9]:
import torchvision.models as model
vgg = model.vgg16(pretrained= True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
#pool 3
#pool 4
#final-output
from torchsummary import summary
summary(vgg,(3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 244, 244]           1,792
              ReLU-2         [-1, 64, 244, 244]               0
            Conv2d-3         [-1, 64, 244, 244]          36,928
              ReLU-4         [-1, 64, 244, 244]               0
         MaxPool2d-5         [-1, 64, 122, 122]               0
            Conv2d-6        [-1, 128, 122, 122]          73,856
              ReLU-7        [-1, 128, 122, 122]               0
            Conv2d-8        [-1, 128, 122, 122]         147,584
              ReLU-9        [-1, 128, 122, 122]               0
        MaxPool2d-10          [-1, 128, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]         295,168
             ReLU-12          [-1, 256, 61, 61]               0
           Conv2d-13          [-1, 256, 61, 61]         590,080
             ReLU-14          [-1, 256,

In [23]:
import torch.nn as nn
from torch.nn import functional as F
from torchvision import models

class FCN8(nn.Module):
  def __init__(self,num_classes=2):
    super(FCN8,self).__init__()
    vgg = models.vgg16(pretrained= True)
    self.features = vgg.features #find the feature map
    #extract intermediate feature map
    self.pool3 = self.features[:17]
    self.pool4 = self.features[:24]
    self.pool5 = self.features
    #1x1 convolutions to convert to class scores
    self.conv1x1_3 = nn.Conv2d(256,num_classes,1)
    self.conv1x1_4 = nn.Conv2d(512,num_classes,1)
    self.conv1x1_5 = nn.Conv2d(512,num_classes,1)
    self.upsample32 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=4,stride=2,padding =1)#2 times
    self.upsample16 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=4,stride=2,padding =1)#2 times
    self.upsample8 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=16,stride=8,padding =4)#8 times

  def forward(self,x):
    pool3_out = self.pool3(x)
    pool4_out = self.pool4(x)
    pool5_out = self.pool5(x)
    conv1x1_3_out = self.conv1x1_3(pool3_out)
    conv1x1_4_out = self.conv1x1_4(pool4_out)
    conv1x1_5_out = self.conv1x1_5(pool5_out)
    x = self.upsample32(conv1x1_5_out) + conv1x1_4_out
    x = self.upsample16(x)+conv1x1_3_out
    x = self.upsample8(x)
    return x

In [24]:
# I= input size
# S = stride
# P = padding
# K = kernel size
# O = (I-1)*S+K-2*P

In [25]:
model = FCN8(num_classes =2)
dummy_input = torch.randn(1,3,224,224)
output = model(dummy_input)
print("Output shape: ",output.shape)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Output shape:  torch.Size([1, 2, 224, 224])
